# Reconnaissance de CV

#### Requirements

In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.0
    Uninstalling typing_extensions-4.12.0:
      Successfully uninstalled typing_extensions-4.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 8.24.0 requires typing-extensions>=4.6; python_version < "3.12", but you have typing-extensions 4.5.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


#### Imports

In [3]:
import os
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi

## Dataset(s)

Deux manières d'import le dataset : avoir les fichiers en local ou le faire a distance (remote)

#### Remote import

In [4]:
# Crée le dossier dataset s'il n'exite pas
!mkdir -p dataset

In [5]:
# Télécharger le dataset dans le dossier 'dataset'
!kaggle datasets download -d mehyarmlaweh/ner-annotated-cvs -p dataset

Dataset URL: https://www.kaggle.com/datasets/mehyarmlaweh/ner-annotated-cvs
License(s): MIT
100%|███████████████████████████████████████▉| 575M/576M [01:08<00:00, 7.55MB/s]
100%|████████████████████████████████████████| 576M/576M [01:08<00:00, 8.86MB/s]


In [6]:
# Décompresser le fichier .zip dans le dossier 'dataset'
!unzip dataset/ner-annotated-cvs.zip -d dataset

Archive:  dataset/ner-annotated-cvs.zip
replace dataset/ResumesJsonAnnotated/ResumesJsonAnnotated/cv (1)_annotated.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [7]:
#RM le zip
!rm dataset/ner-annotated-cvs.zip

In [18]:
import os
import json
import pandas as pd

# Chemin vers le dossier contenant les fichiers JSON
folder_path = 'dataset/ResumesJsonAnnotated/ResumesJsonAnnotated'

# Liste pour stocker les données extraites
data = []
dataskill = []

# Lire chaque fichier JSON dans le dossier
for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
            text = json_data['text']
            annotations = json_data.get('annotations', [])
            skills = []
            for annotation in annotations:
                start, end, label = annotation
                skills.append(label.replace('SKILL: ', '').lower())
                dataskill.append({
                    'file_name': file_name,
                    'text': text,
                    'Skill': label.replace('SKILL: ', '').lower()
                })
            data.append({
                'file_name': file_name,
                'text': text,
                'Skills': skills
            })

# Convertir en DataFrame
df = pd.DataFrame(data)
dfskill = pd.DataFrame(dataskill)

print(df.head())
df


                  file_name  \
0   cv (209)_annotated.json   
1  cv (3108)_annotated.json   
2  cv (4672)_annotated.json   
3  cv (5001)_annotated.json   
4  cv (4872)_annotated.json   

                                                text  \
0        HAMEED SATHIK M\n      HVAC Technical Co...   
1  MD EHTERAJ KHAN\n: 09073357048(M), 0842048419...   
2  INFORMATION TECHNOLOGY CONSULTANT\nHighlights\...   
3  TESSA MAERS\nPROFILE\nA strategic analyst whos...   
4  Titli Foundation \nSystem Developer, Jun 2020-...   

                                              Skills  
0  [building, problem solving, pumps, gmail, java...  
1  [reports, building, inventory, risk, safe, mai...  
2  [reports, reporting, inventory, windows xp, ma...  
3  [matlab, python, vba, business strategy, data ...  
4  [debugging, fixing, identification, detection,...  


,file_name,text,Skills
0,cv (209)_annotated.json,HAMEED SATHIK M\n HVAC Technical Co...,"[building, problem solving, pumps, gmail, java..."
1,cv (3108)_annotated.json,"MD EHTERAJ KHAN\n: 09073357048(M), 0842048419...","[reports, building, inventory, risk, safe, mai..."
2,cv (4672)_annotated.json,INFORMATION TECHNOLOGY CONSULTANT\nHighlights\...,"[reports, reporting, inventory, windows xp, ma..."
3,cv (5001)_annotated.json,TESSA MAERS\nPROFILE\nA strategic analyst whos...,"[matlab, python, vba, business strategy, data ..."
4,cv (4872)_annotated.json,"Titli Foundation \nSystem Developer, Jun 2020-...","[debugging, fixing, identification, detection,..."
...,...,...,...
5024,cv (4574)_annotated.json,ENGINEERING TEACHER\nProfessional Summary\nTo ...,"[reports, building, subject matter, ability, r..."
5025,cv (2508)_annotated.json,MOHAMMED FAIZ UR RAHMAN\nfaizrahman32@gmail.co...,"[gmail, knowledge, plumbing, vr, operations, p..."
5026,cv (2660)_annotated.json,SOFTWARE DEVELOPER MOHAMED AZARUDEEN T H\nth.a...,"[building, gmail, java, knowledge, asp, scorin..."
5027,cv (4981)_annotated.json,"Tata Sons \nAnalyst Support, Apr 2019 - Till d...","[visualisation, python, adobe, managing, deep ..."


In [19]:
print(len(dfskill['Skill'].unique()))
dfskill['Skill'].unique() # nbr de skill

12695


array(['building', 'problem solving', 'pumps', ..., 'construction plans',
       'parking lots', 'time management skills.'], dtype=object)

Developper Backend JS, node.js,sql

In [43]:
df_filtered = dfskill[dfskill['Skill'].str.contains('java', case=False, na=False)] # Teste pour trouver des skills qui correpondent
df_filtered['Skill'].unique()

array(['java', 'core java', 'javascript', 'java script', 'java scripting',
       'java programming', 'java programming language',
       'java development'], dtype=object)

In [26]:
df['prediction'] = 0 # Ajout colonne prediction pour label
df

,file_name,text,Skills,prediction
0,cv (209)_annotated.json,HAMEED SATHIK M\n HVAC Technical Co...,"[building, problem solving, pumps, gmail, java...",0
1,cv (3108)_annotated.json,"MD EHTERAJ KHAN\n: 09073357048(M), 0842048419...","[reports, building, inventory, risk, safe, mai...",0
2,cv (4672)_annotated.json,INFORMATION TECHNOLOGY CONSULTANT\nHighlights\...,"[reports, reporting, inventory, windows xp, ma...",0
3,cv (5001)_annotated.json,TESSA MAERS\nPROFILE\nA strategic analyst whos...,"[matlab, python, vba, business strategy, data ...",0
4,cv (4872)_annotated.json,"Titli Foundation \nSystem Developer, Jun 2020-...","[debugging, fixing, identification, detection,...",0
...,...,...,...,...
5024,cv (4574)_annotated.json,ENGINEERING TEACHER\nProfessional Summary\nTo ...,"[reports, building, subject matter, ability, r...",0
5025,cv (2508)_annotated.json,MOHAMMED FAIZ UR RAHMAN\nfaizrahman32@gmail.co...,"[gmail, knowledge, plumbing, vr, operations, p...",0
5026,cv (2660)_annotated.json,SOFTWARE DEVELOPER MOHAMED AZARUDEEN T H\nth.a...,"[building, gmail, java, knowledge, asp, scorin...",0
5027,cv (4981)_annotated.json,"Tata Sons \nAnalyst Support, Apr 2019 - Till d...","[visualisation, python, adobe, managing, deep ...",0


In [66]:
#List = ['blockchain', 'architecture solutions','program development', 'project development']

df_filtered = dfskill[dfskill['Skill'].str.contains(r'(js|javascript|java script)', case=False, na=False)]
df_filtered = df[df['file_name'].isin(df_filtered['file_name'])]
df['prediction'] = df['file_name'].isin(df_filtered['file_name']).astype(int) # Mettre prediction sur 1 pour les file_names avec javascript

test = df[df['prediction'] == 1]
print(len(test))
for skill in test['Skills']: # Afficher les skills des CV interressant
    print(skill)

127
['reports', 'building', 'inventory', 'risk', 'safe', 'maintaining', 'gmail', 'java', 'knowledge', 'plumbing', 'capex', 'asp', 'contracts', 'strategies', 'operations', 'social networking', 'team management', 'performance', 'managing', 'technology', 'savings', 'operating systems', 'core java', 'facilities management', 'detection', 'service delivery', 'budgets', 'performance management', 'general administration', 'vendors', 'skills', 'compliance', 'opex', 'scripting language', 'projects', 'new processes', 'deploying', 'process', 'sql server', 'c', 'passport', 'business stakeholders', 'functions', 'hvac systems', 'developing', 'supervising', 'fire', 'trends', 'networking', 'voice', 'science', 'location', 'management', 'implementation', 'counseling', 'appreciation', 'reliable', 'interpersonal skills', 'ownership', 'delta', 'staffing', 'building services', 'ms access', 'levels', 'access', 'hvac', 'm', 'cctv', 'it skills', 'expansion', 'bms', 'processes', 'oracle', 'delivery', 'team', 'an

/tmp/ipykernel_16365/1688187480.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_filtered = dfskill[dfskill['Skill'].str.contains(r'(js|javascript|java script)', case=False, na=False)]


In [67]:
df

,file_name,text,Skills,prediction
0,cv (209)_annotated.json,HAMEED SATHIK M\n HVAC Technical Co...,"[building, problem solving, pumps, gmail, java...",0
1,cv (3108)_annotated.json,"MD EHTERAJ KHAN\n: 09073357048(M), 0842048419...","[reports, building, inventory, risk, safe, mai...",1
2,cv (4672)_annotated.json,INFORMATION TECHNOLOGY CONSULTANT\nHighlights\...,"[reports, reporting, inventory, windows xp, ma...",0
3,cv (5001)_annotated.json,TESSA MAERS\nPROFILE\nA strategic analyst whos...,"[matlab, python, vba, business strategy, data ...",0
4,cv (4872)_annotated.json,"Titli Foundation \nSystem Developer, Jun 2020-...","[debugging, fixing, identification, detection,...",0
...,...,...,...,...
5024,cv (4574)_annotated.json,ENGINEERING TEACHER\nProfessional Summary\nTo ...,"[reports, building, subject matter, ability, r...",0
5025,cv (2508)_annotated.json,MOHAMMED FAIZ UR RAHMAN\nfaizrahman32@gmail.co...,"[gmail, knowledge, plumbing, vr, operations, p...",0
5026,cv (2660)_annotated.json,SOFTWARE DEVELOPER MOHAMED AZARUDEEN T H\nth.a...,"[building, gmail, java, knowledge, asp, scorin...",1
5027,cv (4981)_annotated.json,"Tata Sons \nAnalyst Support, Apr 2019 - Till d...","[visualisation, python, adobe, managing, deep ...",0


In [118]:
one = df[df['prediction'] == 1]
zero = df[df['prediction'] == 0]
zero = zero.sample(n=len(one), random_state=42)
dfModel = pd.concat([one, zero])
dfModel = dfModel.sample(frac=1, random_state=42).reset_index(drop=True) # Dataset plus petit pour meilleur ration entre prediction = 0 ou 1
dfModel

,file_name,text,Skills,prediction
0,cv (188)_annotated.json,Curricul...,"[reports, building, deployment, problem solvin...",0
1,cv (1398)_annotated.json,NADUKKARA HOUSE \n ...,"[reports, matlab, gmail, knowledge, arts, age,...",1
2,cv (893)_annotated.json,LOREN SHEVITZ\n345 W. FULLERTON #2101 PHONE: 7...,"[ability, python, java, arts, tcp/ip, ms offic...",1
3,cv (2097)_annotated.json,...,"[gmail, knowledge, marketing, resume, strength...",0
4,cv (1405)_annotated.json,\nCURRICULUM VITAE \nPERSONAL DETAILS \nNAM...,"[building, ability, prioritize, problem solvin...",1
...,...,...,...,...
249,cv (2249)_annotated.json,Deepak Saini\nE-mail: sainideepak142@gmail.com...,"[ability, gmail, java, knowledge, certificatio...",1
250,cv (4720)_annotated.json,CONSULTANT\nCareer Overview\nVidya is a Gradua...,"[reports, reporting, passion, ability, sales a...",1
251,cv (4927)_annotated.json,"ASI Technologies \nIntern, \nDec 2020 - Ongoin...","[building, deployment, python, web development...",1
252,cv (3703)_annotated.json,CURRICULAM-VITAE\nABDULLAH\nWard No. 30 Mohall...,"[reports, building, ability, gmail, knowledge,...",0


In [119]:
dfModel = dfskill[dfskill['file_name'].isin(dfModel['file_name'])]
print(len(dfModel['Skill'].unique()))
print(dfModel['Skill'].unique())
dfModel = dfModel.merge(df[['file_name', 'prediction']], on='file_name', how='left') # On sépare les skills
dfModel

4985
['reports' 'building' 'inventory' ... 'multiple tasks' 'queues' 'ipad']


,file_name,text,Skill,prediction
0,cv (3108)_annotated.json,"MD EHTERAJ KHAN\n: 09073357048(M), 0842048419...",reports,1
1,cv (3108)_annotated.json,"MD EHTERAJ KHAN\n: 09073357048(M), 0842048419...",building,1
2,cv (3108)_annotated.json,"MD EHTERAJ KHAN\n: 09073357048(M), 0842048419...",inventory,1
3,cv (3108)_annotated.json,"MD EHTERAJ KHAN\n: 09073357048(M), 0842048419...",risk,1
4,cv (3108)_annotated.json,"MD EHTERAJ KHAN\n: 09073357048(M), 0842048419...",safe,1
...,...,...,...,...
32280,cv (2660)_annotated.json,SOFTWARE DEVELOPER MOHAMED AZARUDEEN T H\nth.a...,project,1
32281,cv (2660)_annotated.json,SOFTWARE DEVELOPER MOHAMED AZARUDEEN T H\nth.a...,personal,1
32282,cv (2660)_annotated.json,SOFTWARE DEVELOPER MOHAMED AZARUDEEN T H\nth.a...,applications,1
32283,cv (2660)_annotated.json,SOFTWARE DEVELOPER MOHAMED AZARUDEEN T H\nth.a...,airline,1


In [120]:
if 'text' in dfModel.columns:
    dfModel = dfModel.drop(columns=['text'])

df_pivot = dfModel.pivot_table(index='file_name', columns='Skill', aggfunc='size', fill_value=0).astype(bool)
df_pivot = df_pivot.rename(columns={'prediction': 'prediction_bool'})


df_prediction = dfModel[['file_name', 'prediction']].drop_duplicates().set_index('file_name')
dfModel = df_pivot.join(df_prediction)

dfModel = dfModel.reset_index()
dfModel
# On renverse notre dataframe où chaque colonne est un skill 

,file_name,.net,.net technologies,1 year experience,10 years experience,10 years of experience,12 months,12c,15 years of experience,2 years experience,...,xml,xml messages,xp,xpath,xquery,xsd,xslt,zend,zfs,prediction
0,cv (1021)_annotated.json,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
1,cv (1022)_annotated.json,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,1
2,cv (1034)_annotated.json,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,1
3,cv (1036)_annotated.json,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
4,cv (1059)_annotated.json,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,cv (887)_annotated.json,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
247,cv (893)_annotated.json,False,False,False,False,False,False,False,False,False,...,True,False,True,False,False,False,False,False,False,1
248,cv (934)_annotated.json,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,1
249,cv (935)_annotated.json,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,1


In [121]:
len(dfModel[dfModel['prediction'] == 1]) # On retrouve bien nos 127CV prediction = 1

127

In [125]:
from sklearn.model_selection import train_test_split

dfModel = dfModel.drop('file_name', axis=1)
X = dfModel.drop('prediction', axis=1)
y = dfModel['prediction']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42) # Seprare en jeu de training et jeu de validation

print(f'Taille de lentrainement : {X_train.shape[0]}')
print(f'Taille de validation : {X_val.shape[0]}')


Taille de lentrainement : 200
Taille de validation : 51


In [126]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
y_pred_proba = model.predict_proba(X_val)[:, 1]

accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred_proba)
classification_rep = classification_report(y_val, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'ROC AUC Score: {roc_auc:.4f}')
print('Classification Report:')
print(classification_rep) # On Applique RandomForest dessus

Accuracy: 0.9412
ROC AUC Score: 0.9938
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.92      0.94        25
           1       0.93      0.96      0.94        26

    accuracy                           0.94        51
   macro avg       0.94      0.94      0.94        51
weighted avg       0.94      0.94      0.94        51



Cree nouvelle branche
Choisir type de metier voulu
labeliser avec que les skills
premier modèle


AMelioration : nlp sur annonce pour detecter skill

In [9]:
# Prérequis : Dans la section "API", cliquez sur "Create New API Token".
# Cela téléchargera un fichier kaggle.json contenant vos identifiants API.

# Example d'import
os.environ['KAGGLE_USERNAME'] = 'your_kaggle_username'
os.environ['KAGGLE_KEY'] = 'your_kaggle_key'

api = KaggleApi()
api.authenticate()
api.dataset_download_files('heptapod/titanic', path='.', unzip=True)

df = pd.read_csv('titanic/train.csv')
print(df.head())


Dataset URL: https://www.kaggle.com/datasets/heptapod/titanic


FileNotFoundError: [Errno 2] No such file or directory: 'titanic/train.csv'

#### Local import

In [ ]:
file_path = 'data/titanic.csv'

df = pd.read_csv(file_path)

print(df.head())


## Modele Lineaire 

#### Architecture

#### Training

#### Test

## Arbre de decision

#### Architecture

#### Training

#### Test